In [1]:
!pip install jiwer -qq

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [4]:
from predict import predict
import onnxruntime
import onnx
from pathlib import Path
import jiwer
import pandas as pd
from utils import get_files
from pdb import set_trace
from words2num import normalize
pd.set_option('max_colwidth', 400)

In [5]:
model = onnx.load("/content/output.onnx")
model = onnxruntime.InferenceSession(model.SerializeToString())

In [6]:
data_path = "/content/test-bahasa/"

In [7]:
txts = get_files(data_path,[".txt"],recurse=True)
txts

(#780) [PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ118-000005.txt'),PosixPath('/content/test-bahasa/iium-audio-iium-21.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-oleh huruf iaitu dan.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-bagaimanapun istilah melayu adalah.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ094-000005.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ223-000005.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-pertama acara a reveal.txt'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-43.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-tolong-sebut-tidak sopan.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ137-000005.txt')...]

In [8]:
# find data with number in it
nombor = ["satu","dua",'tiga','puluh','ratus','ribu']
filtered_wavs = []
for i in txts:
    text = i.read_text().split()
    got = any([i in nombor for i in text])
    if got:
        filtered_wavs.append(i.with_suffix(".wav"))

# Predict single or multiple file

In [9]:
preds,files,entropy = predict(model,filtered_wavs)

100%|██████████| 37/37 [00:15<00:00,  2.43it/s]


In [10]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([label,i,normalize(i)])

In [11]:
df = pd.DataFrame(data)
df.columns = ["label","pred","norm_pred"]
df.head(10)

,label,pred,norm_pred
0,gila cape banget kecepatan lari kamu kalau lagi panik nambah dua kali,kilah capit bangyit kecapatan lari kamu kalau lagi panik enambah dua kali,kilah capit bangyit kecapatan lari kamu kalau lagi panik enambah 2 kali
1,pengalaman aku aku satu geng nak makan tengahari beli makanan aku ambik lauk telur jela nak jimat kawan aku ini ambik lauk ayam,pengalaman aku aku satu geng nak makan tengah hari boleh makanan aku ambil laut telur jelar,pengalaman aku aku 1 geng nak makan tengah hari boleh makanan aku ambil laut telur jelar
2,aku ada dua orang sahabat baik,aku ada dua orang sahabat baik,aku ada 2 orang sahabat baik
3,kelihatan selepas tujuh puluh dua jam daripada aktiviti tersebut atau selepas,kelihatan selepas tujuh puluh dua jam daripada aktiviti tersebut atau selepas,kelihatan selepas 72 jam daripada aktiviti tersebut atau selepas
4,bandung dua ribu tujuh belas,bandung dua ribu tujuh belas,bandung 2007 belas
5,studi ini juga mendapati sebanyak empat puluh tiga percent anak yang datang,studi ini juga mendapati sebanyak empat puluh tiga persen anak yang datang,studi ini juga mendapati sebanyak 43 persen anak yang datang
6,bayangkan sudah lah ibu tunggal anak dua laki pulak makin bermasalah ketika itu,bayangkan sudahlai ibu tunggal anak dua lelaki,bayangkan sudahlai ibu tunggal anak 2 lelaki
7,sama kalau kekasih kau ikut jantan lain kan saving duit hantaran dua puluh k kau tu,sama kalau ke kasih kau ikut jantan lainkan sering duit hantara dua puluh ke kau tu,sama kalau ke kasih kau ikut jantan lainkan sering duit hantara 20 ke kau tu
8,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,45 ahli belia yang dilantik itu menjadi penyambung legasi
9,tertuju pada gerombolan laki laki di koridor kelas dua belas ipa enam sedang berkumpul,tertuju pada grumbulan lelaki laki di koridor kelas dua belas ip enam sedang,tertuju pada grumbulan lelaki laki di koridor kelas 2 belas ip 6 sedang


In [13]:
# word to num normalization still wip, let use know if found a bug so we can fix it asap
# TODO: fix belas word